In [29]:
import pandas as pd
import urllib.request
# Read an excel file into a variable named df
df = pd.read_excel('24996-store-overview.xlsx')
df.head()


,Location,Latitude,Longitude,Population,Url,Costs Year 2014,Costs Year 2015,Costs Year 2016,Costs Year 2017,Costs Year 2018,Costs Year 2019,Costs Year 2020,Costs Year 2021,Costs Year 2022
0,Alingsås,57.930020,12.536211,27501,https://cda.hhs.se/toystorey/24996-alings%C3%A...,15900,17100,16600,16900,16400,16900,17000,17300,15300
1,Boden,65.825119,21.688703,16848,https://cda.hhs.se/toystorey/24996-boden.xlsx,15200,16300,17100,17500,18000,18200,18500,18600,16600
2,Gävle,60.674880,17.141273,77699,https://cda.hhs.se/toystorey/24996-g%C3%A4vle....,16400,17500,18200,18100,18200,18700,19000,19100,17200
3,Göteborg,57.708870,11.974560,600473,https://cda.hhs.se/toystorey/24996-g%C3%B6tebo...,21300,22000,21400,21800,21900,22100,22200,21900,20700
4,Helsingborg,56.046467,12.694512,112496,https://cda.hhs.se/toystorey/24996-helsingborg...,18300,19100,19100,18900,19000,18900,18700,19100,16900


In [ ]:
# Step 1 – Downloading data files

# iterate over every row in the DataFrame
for index, row in df.iterrows():
    urllib.request.urlretrieve(row['Url'], row['Location']+".xlsx")

In [62]:
# Step 2 – Streamlining the downloaded files
df = pd.read_excel('output/Mordor.xlsx')
df.head()
# =INDEX($I$1:$R$1, MATCH("OK", I2:R2)) and remove unnesnsary columns
    

,receipt_product_id,product_id,receipt_id,is_return,quantity,location,date,payment_method,sales_rep
0,dcd1c93d-2691-4c10-b0d6-4eaad1b71ebb,PR4938,11,True,1.0,Mordor,2014-04-21 23:38:04,American Express,Gilderoy Lockhart
1,06510aeb-125d-45db-b7c3-8f08b1636ddc,PR5625,11,True,2.0,Mordor,2014-04-21 23:38:04,American Express,Gilderoy Lockhart
2,4b8b4467-d40d-4bd4-a3da-461a9127e443,PR6601,11,True,1.0,Mordor,2014-04-21 23:38:04,American Express,Gilderoy Lockhart
3,f38b4f7d-17af-4c12-af8c-5622e8deb7b7,PR6212,11,True,7.0,Mordor,2014-04-21 23:38:04,American Express,Gilderoy Lockhart
4,d13adb31-3452-47e4-847a-0ae824c4bc3c,PR2186,11,True,4.0,Mordor,2014-04-21 23:38:04,American Express,Gilderoy Lockhart


In [72]:
# Step 3 – Loading everything into a DataFrame

import glob
#glob.glob returns a list of all files in this particular folder

# Create list of DataFrames for each city w list comprehension
# Loop over the files in the output folder and create a DataFrame for each file and add it to a list.
DF_list = [pd.read_excel(city) for city in glob.glob('output/*')] 


In [82]:
# Step 4 – Fetching data from MySql

#You may need to install PyMysql to be able to run this
from sqlalchemy import create_engine
host = "mysql-1.cda.hhs.se"
username = "24996"
password = "toystorey"
schema = "ToyStoreySales"
# format replaces {} in the string with the respective inputs to the function
connection_string = "mysql+pymysql://{}:{}@{}/{}".format(username, password, host,
schema)
# this creates a connection from the string we just created
connection = create_engine(connection_string)
# use the connection to run SQL query and store it into the DataFrame variable df

df_RP_R = pd.read_sql("SELECT * FROM ReceiptProduct RP LEFT JOIN Receipt R ON RP.receipt_id = R.receipt_id", con=connection)
df_RP = pd.read_sql("SELECT * FROM ReceiptProduct RP", con=connection)
df_R = pd.read_sql("SELECT * FROM Receipt", con=connection)
df_P = pd.read_sql("SELECT * FROM Product", con=connection)

In [89]:
# Step 5 – Processing the data

# Add location
df_RP_R["location"] = "internet"
df_RP_R["sales_rep"] = "internet"

# remove discount code
# remove home delivery
# remove customer id
del df_RP_R["discount_code"]
del df_RP_R["home_delivery"]
del df_RP_R["customer_id"]

# remove one reciepie id
df_RP_R.drop(df.columns[5], axis=1, inplace=True)



KeyError: 'discount_code'

In [103]:
# Step 5 – Processing the data

# Add all into one dataframe:
df = df_RP_R
for d in DF_list:
    df = pd.concat([df, d])

In [156]:
# Step 6 – Finding frauds

output = df.groupby(['sales_rep']).sum().reset_index()
output["return_rate"] = (output['is_return'] / output['quantity']).round(2)
output.sort_values(by="return_rate", ascending=False, inplace=True)

# Get all receipts from Gilderoy Lockhart and Livia Månsson (return rate > 0.05)
# doing this to not have to join everything.
df_gilderoy = df.loc[df['sales_rep'] == "Gilderoy Lockhart"]
df_livia= df.loc[df['sales_rep'] == "livia månsson"]

#print(df_gilderoy)

#TODO ERROR PRICE VALUES DOES NOT EXIST FOR ALL RECIEPT_PRODUCT_IDS.
df_price = pd.merge(df_RP, df_P, on="product_id", how="left")
df_sum = pd.merge(df_gilderoy, df_price, on="receipt_product_id", how="left")
df_sum = df_sum.loc[df_sum['is_return_x'] >= 1]
df_sum["returned_revenue"] = df_sum['quantity_x']*df_sum['price']
df_sum.head()
#returned_sum = df_sum.sum().reset_index()
#print(returned_sum)

df_price.loc[df_price['receipt_product_id'] == "dcd1c93d-2691-4c10-b0d6-4eaad1b71ebb"]

,receipt_product_id,receipt_id,product_id,quantity,is_return,product_name,manufacturer,category,price,cost
